In [3]:
import torch

In [2]:
class ModelTrain():
    def __init__(self, config):
        self.config = config

    def train_Seq2Seq(self, model, optimizer_model, loss_function, real_train, missing_train, mask_train, step_per_epoch, helper):
    
        loss_evol = []
        real_dataset = torch.empty((0, self.config.input_size), dtype=torch.float32).to(self.config.device)
        gen_dataset = torch.empty((0, self.config.input_size), dtype=torch.float32).to(self.config.device)
        mask = torch.empty((0, self.config.input_size), dtype=torch.float32).to(self.config.device)
        epoch = 300
        
        for i in range(epoch):
            count = 0
            epoch_loss = 0.0

            for step in range(step_per_epoch):
                # Data for training the discriminator
                real_data, real_label = helper.gen_real_batch(self.config.batch_size, count, real_train)

                z_input, mask_input = helper.gen_z_input(self.config.batch_size, count, missing_train, mask_train)
                #z_input = torch.cat((z_input, (1 - mask_input[:, :, 1]).unsqueeze(2)), dim=2)
                model.zero_grad()
                generated_samples = model(z_input)

                L2 = torch.sum((real_data - generated_samples)**2)
                #L3 = torch.sum((real_data[:, :, self.config.column] - generated_samples[:, :, self.config.column])**2)
                g_loss = loss_function(generated_samples, real_data) + L2
                g_loss.backward()
                optimizer_model.step()
                epoch_loss += g_loss.item()
                
                count += 1
                if i == epoch - 1:
                    real_dataset = torch.cat([real_dataset, real_data[:, -1, :]], dim=0)
                    gen_dataset = torch.cat([gen_dataset, generated_samples[:, -1, :]], dim=0)
                    mask = torch.cat([mask, mask_input[:, -1, :]], dim=0)

            epoch_loss /= step_per_epoch # average loss per batch
            loss_evol.append(epoch_loss)
            
            print(f"epoch: {i}, Train loss: {epoch_loss:.7f}")

        return real_dataset, gen_dataset, loss_evol, mask

In [3]:
class ModelTest():
    def __init__(self, config):
        self.config = config

    def test_model(self, model, real_data_test, missing_data_test, mask_test, loss_function, step_per_epoch, helper):
        model.eval()
        test_loss = []

        # batch_size = real_data_test.shape[0]
        # print(batch_size)
        real_dataset = torch.empty((0, self.config.input_size), dtype=torch.float32).to(self.config.device)
        gen_dataset = torch.empty((0, self.config.input_size), dtype=torch.float32).to(self.config.device)
        mask = torch.empty((0, self.config.input_size), dtype=torch.float32).to(self.config.device)
        with torch.no_grad():
            count = 0
            for step in range(step_per_epoch):
                real_data, real_label = helper.gen_real_batch(self.config.batch_size, count, real_data_test)

                z_input, mask_input = helper.gen_z_input(self.config.batch_size, count, missing_data_test, mask_test)
                
                imputed_results = model(z_input)

                #L2 = torch.sum((real_data - imputed_results)**2)
                L2 = torch.sum((real_data[:, :, self.config.column] - imputed_results[:, :, self.config.column])**2)
                loss = loss_function(imputed_results, real_data) + L2
                test_loss.append(loss.item())

                real_dataset = torch.cat([real_dataset, real_data[:, -1, :]], dim=0)
                gen_dataset = torch.cat([gen_dataset, imputed_results[:, -1, :]], dim=0)
                mask = torch.cat([mask, mask_input[:, -1, :]], dim=0)

                count += 1

        return real_dataset, gen_dataset, test_loss, mask